In [1]:
!pip install keras==asdf

  Could not find a version that satisfies the requirement keras==asdf (from versions: 0.2.0, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.0.8, 2.0.9, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.1.5, 2.1.6, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.2.4)
No matching distribution found for keras==asdf


In [2]:
!pip install keras==2.0.1
!pip install git+https://github.com/bstriner/keras-adversarial.git

  Cloning https://github.com/bstriner/keras-adversarial.git to /tmp/pip-req-build-klz9ds9q
  Running setup.py bdist_wheel for keras-adversarial ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-mboalvrp/wheels/f4/34/ba/6041a92244597803d1f8954d649e7899d011d46f33c02dc476
Successfully built keras-adversarial


In [3]:
## image utility methods

import numpy as np
import keras.backend as K
from keras.layers import Input, Reshape


def dim_ordering_fix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 2, 3, 1))


def dim_ordering_unfix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 3, 1, 2))


def dim_ordering_shape(input_shape):
    if K.image_dim_ordering() == 'th':
        return input_shape
    else:
        return (input_shape[1], input_shape[2], input_shape[0])


def dim_ordering_input(input_shape, name):
    if K.image_dim_ordering() == 'th':
        return Input(input_shape, name=name)
    else:
        return Input((input_shape[1], input_shape[2], input_shape[0]), name=name)


def dim_ordering_reshape(k, w, **kwargs):
    if K.image_dim_ordering() == 'th':
        return Reshape((k, w, w), **kwargs)
    else:
        return Reshape((w, w, k), **kwargs)


def channel_axis():
    if K.image_dim_ordering() == 'th':
        return 1
    else:
        return 3

Using TensorFlow backend.


In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import keras.backend as K
from keras.layers import Flatten, Dropout, LeakyReLU, Input, Activation, Dense, BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras_adversarial.image_grid_callback import ImageGridCallback
from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling

In [0]:
def model_generator():
    nch = 256
    g_input = Input(shape=[100])
    H = Dense(nch * 14 * 14)(g_input)
    H = BatchNormalization()(H)
    H = Activation("relu")(H)
    H = dim_ordering_reshape(nch, 14)(H)
    H = UpSampling2D(size=(2, 2))(H)
    H = Conv2D(int(nch / 2), (3, 3), padding="same")(H)
    H = BatchNormalization()(H)
    H = Activation("relu")(H)
    H = Conv2D(int(nch / 4), (3, 3), padding="same")(H)
    H = BatchNormalization()(H)
    H = Activation("relu")(H)
    H = Conv2D(1, (1, 1), padding="same")(H)
    g_V = Activation("sigmoid")(H)
    return Model(g_input, g_V)

In [0]:
def model_discriminator(input_shape=(1, 28, 28), dropout_rate=0.5):
    d_input = dim_ordering_input(input_shape, name="input_x")
    nch = 512
    # nch = 128
    H = Conv2D(int(nch / 2), (5, 5),
               strides=(2, 2),
               padding="same",
               activation="relu",
               )(d_input)
    H = LeakyReLU(0.2)(H)
    H = Dropout(dropout_rate)(H)
    H = Conv2D(nch, (5, 5),
               strides=(2, 2),
               padding="same",
               activation="relu",
               )(H)
    H = LeakyReLU(0.2)(H)
    H = Dropout(dropout_rate)(H)
    H = Flatten()(H)
    H = Dense(int(nch / 2))(H)
    H = LeakyReLU(0.2)(H)
    H = Dropout(dropout_rate)(H)
    d_V = Dense(1, activation="sigmoid")(H)
    return Model(d_input, d_V)

In [0]:
def mnist_process(x):
    x = x.astype(np.float32) / 255.0
    return x


def mnist_data():
    (xtrain, ytrain), (xtest, ytest) = mnist.load_data()
    return mnist_process(xtrain), mnist_process(xtest)


def generator_sampler(latent_dim, generator):
    def fun():
        zsamples = np.random.normal(size=(10 * 10, latent_dim))
        gen = dim_ordering_unfix(generator.predict(zsamples))
        return gen.reshape((10, 10, 28, 28))

    return fun

In [8]:
# z \in R^100
latent_dim = 100
# x \in R^{28x28}
input_shape = (1, 28, 28)

# generator (z -> x)
generator = model_generator()
# discriminator (x -> y)
discriminator = model_discriminator(input_shape=input_shape)
# gan (x - > yfake, yreal), z generated on GPU
gan = simple_gan(generator, discriminator,
                 normal_latent_sampling((latent_dim,)))

# print summary of models
generator.summary()
discriminator.summary()
gan.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50176)             5067776   
_________________________________________________________________
batch_normalization_1 (Batch (None, 50176)             200704    
_________________________________________________________________
activation_1 (Activation)    (None, 50176)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 14, 14, 256)       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 256)       0         
_________________________________________________________________
con

In [9]:
# build adversarial model
model = AdversarialModel(base_model=gan,
                         player_params=[generator.trainable_weights, 
                                        discriminator.trainable_weights],
                         player_names=["generator", "discriminator"])
model.adversarial_compile(adversarial_optimizer=AdversarialOptimizerSimultaneous(),
                          player_optimizers=[Adam(1e-4, decay=1e-4),
                                             Adam(1e-3, decay=1e-4)],
                          loss="binary_crossentropy")

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
# train model
generator_cb = ImageGridCallback("output/gan_convolutional/epoch-{:03d}.png",
                                 generator_sampler(latent_dim, generator))
callbacks = [generator_cb]
if K.backend() == "tensorflow":
    callbacks.append(
        TensorBoard(log_dir=os.path.join("output/gan_convolutional/", "logs/"),
                    histogram_freq=0, write_graph=True, write_images=True))

xtrain, xtest = mnist_data()
xtrain = dim_ordering_fix(xtrain.reshape((-1, 1, 28, 28)))
xtest = dim_ordering_fix(xtest.reshape((-1, 1, 28, 28)))
y = gan_targets(xtrain.shape[0])
ytest = gan_targets(xtest.shape[0])
history = model.fit(x=xtrain, y=y, validation_data=(xtest, ytest), 
                    callbacks=[generator_cb], epochs=100,
                    batch_size=32)
df = pd.DataFrame(history.history)
df.to_csv("output/gan_convolutional/history.csv")

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
 8224/60000 [===>..........................] - ETA: 272s - loss: 22.1019 - generator_loss: 22.0093 - generator_yfake_loss: 12.2842 - generator_yreal_loss: 9.7251 - discriminator_loss: 0.0926 - discriminator_yfake_loss: 0.0526 - discriminator_yreal_loss: 0.0399

KeyboardInterrupt: ignored

In [0]:
generator.save("output/gan_convolutional/generator.h5")
discriminator.save("output/gan_convolutional/discriminator.h5")